Imports

In [11]:
from __future__ import print_function
import argparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.autograd import Variable
from typing import Tuple
from torch.utils.data import Dataset,DataLoader
from sklearn.metrics import confusion_matrix, top_k_accuracy_score
import torchvision                                                       
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.transforms import ToTensor
from torch import nn, optim

Prepare datasets

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def get_data(batch_size: int) -> Tuple[DataLoader, DataLoader]:
    
    train_dataset = torchvision.datasets.FashionMNIST(root="./",train=True,transform= ToTensor())

    test_dataset = torchvision.datasets.FashionMNIST(root="./", transform = ToTensor(), train=False)    

    train_DL = DataLoader(train_dataset, batch_size=batch_size)
    test_DL = DataLoader(test_dataset, batch_size=batch_size)
    return (train_DL, test_DL)

Define Network

In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        """ self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10) """
        
        self.model = models.resnet50(pretrained=True)
        self.model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 10)
        
    def forward(self, x):
        return self.model(x)
    
fmnist_model = Net()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\tarip\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\tarip\AppData\Local\Temp\ipykernel_30100\4109322026.py", line 20, in <module>
    fmnist_model = Net()
  File "C:\Users\tarip\AppData\Local\Temp\ipykernel_30100\4109322026.py", line 11, in __init__
    self.model = models.resnet50(pretrained=True)
NameError: name 'models' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\tarip\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\tarip\AppData\Loca

Run and test model

In [10]:
fmnist_model.to(device)

error = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fmnist_model.parameters(), lr=.001)

count = 0

epochs = 5
batch_size = 200

data_set = get_data(200)

loss_list = []
iteration_list = []
accuracy_list = []

# Lists for knowing classwise accuracy
predictions_list = []
labels_list = []
for epoch in range(epochs):
    for images, labels in data_set[0]:
        # Transfering images and labels to GPU if available
        images, labels = images.to(device), labels.to(device)
    
        train = Variable(images.view(200, 1, 28, 28))
        labels = Variable(labels)
        # Forward pass 
        outputs = fmnist_model(train)
        loss = error(outputs, labels)
        
        # Initializing a gradient as 0 so there is no mixing of gradient among the batches
        optimizer.zero_grad()
        
        #Propagating the error backward
        loss.backward()
        
        # Optimizing the parameters
        optimizer.step()
    
        count += 1
    
    # Testing the model
    
        if not (count % 50):    # It's same as "if count % 50 == 0"
            total = 0
            correct = 0
        
            for images, labels in data_set[1]:
                images, labels = images.to(device), labels.to(device)
                labels_list.append(labels)
            
                test = Variable(images.view(200, 1, 28, 28))
            
                outputs = fmnist_model(test)
            
                predictions = torch.max(outputs, 1)[1].to(device)
                predictions_list.append(predictions)
                correct += (predictions == labels).sum()
            
                total += len(labels)
            
            accuracy = correct * 100 / total
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
        
        if not (count % 500):
            print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, loss.data, accuracy))
            



Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\tarip\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\tarip\AppData\Local\Temp\ipykernel_30100\2682356053.py", line 1, in <module>
    fmnist_model.to(device)
NameError: name 'fmnist_model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\tarip\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\tarip\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\ultratb.py",